In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ecommerce-behavior-data-from-multi-category-store/2019-Nov.csv
/kaggle/input/ecommerce-behavior-data-from-multi-category-store/2019-Oct.csv


In [9]:
path = "/kaggle/input/ecommerce-behavior-data-from-multi-category-store/2019-Oct.csv"


# Pandas (chunksize)

In [34]:
import pandas as pd
import time

start = time.time()

chunksize = 50000  
chunks = []

for chunk in pd.read_csv(path, chunksize=chunksize):
    chunks.append(chunk)

df = pd.read_csv(path, nrows=100000)

end = time.time()
print(f" تم تحميل الملف بنجاح في {end - start:.2f} ثانية")
print(f"عدد الأسطر الإجمالي: {len(df)}")


 تم تحميل الملف بنجاح في 85.09 ثانية
عدد الأسطر الإجمالي: 100000


In [36]:
memory = df.memory_usage(deep=True).sum() / (1024**2)

# Dask 

In [12]:
import dask.dataframe as dd
import time

start_dask = time.time()

df_dask = dd.read_csv(path).compute()


end_dask = time.time()
print(f" الزمن المستغرق: {end_dask - start_dask:.2f} ثانية")
print(f"عدد الأسطر الإجمالي: {len(df_dask)}")

 الزمن المستغرق: 99.83 ثانية
عدد الأسطر الإجمالي: 42448764


In [13]:
memory_dask = df_dask.memory_usage(deep=True).sum() / (1024 ** 2)

# Compressed (Gzip) 

In [25]:
!cp /kaggle/input/ecommerce-behavior-data-from-multi-category-store/2019-Oct.csv /kaggle/working/


In [26]:
!gzip /kaggle/working/2019-Oct.csv


In [16]:
start_time_gzip = time.time()
df_gzip = pd.read_csv('/kaggle/working/2019-Oct.csv.gz', compression='gzip')
end_time_gzip = time.time()
print(f" الزمن المستغرق: {end_time_gzip - start_time_gzip:.2f} ثانية")
print(f"عدد الأسطر الإجمالي: {len(df_gzip)}")

 الزمن المستغرق: 149.84 ثانية
عدد الأسطر الإجمالي: 42448764


In [28]:
import os

file_path = '/kaggle/working/2019-Oct.csv.gz'

# الحصول على الحجم بالبايت
file_size_bytes = os.path.getsize(file_path)

# تحويل إلى ميغابايت
memory_gzip = file_size_bytes / (1024 ** 2)


# comparison

In [37]:
data = {
    "Method": ["Pandas (chunksize)", "Dask", "Compressed (Gzip)"],
    "Time (seconds)": [end - start, end_dask - start_dask, end_time_gzip - start_time_gzip],
    "Total Memory Used (MB)": [memory, memory_dask, memory_gzip]
}
df_comparison = pd.DataFrame(data)
print("\nThe comparison:\n")
print(df_comparison.to_string(index=False))


The comparison:

            Method  Time (seconds)  Total Memory Used (MB)
Pandas (chunksize)       85.085593               37.072811
              Dask       99.828463             6627.627619
 Compressed (Gzip)      149.839266             1669.942356


###  التحليل 

####  من حيث الزمن (الأداء):
- **Pandas (chunksize)** كانت **الأسرع** بمتوسط **85 ثانية فقط**.  
  السبب هو أن هذه الطريقة تقوم بتحميل البيانات على دفعات صغيرة (chunks)، مما يقلل من الحمل على الذاكرة أثناء المعالجة.

- **Dask** استغرق وقتاً أطول (حوالي **100 ثانية**)، رغم كونه يعمل بالتوازي، إلا أن **إدارة المهام المتعددة (overhead)** تسبب بعض البطء في المهام الصغيرة.

- **Gzip** كان **الأبطأ** لأن عملية **فك الضغط أثناء القراءة** تتطلب وقتاً إضافياً.

---

####  من حيث استهلاك الذاكرة:
- **Pandas (chunksize)** كانت **الأفضل** باستهلاك منخفض جداً (≈ **37 MB**).  
  وهذا يجعلها مناسبة للأجهزة **محدودة الذاكرة**.

- **Dask** استخدم **أعلى كمية من الذاكرة** (≈ **6.6 GB**) بسبب **تقسيم البيانات إلى مهام متعددة** تُخزن مؤقتاً في الذاكرة أثناء المعالجة.

- **Gzip** استهلك **ذاكرة كبيرة نسبياً** (≈ **1.6 GB**) نتيجة **فك الضغط المؤقت للبيانات** أثناء التحميل.
